In [1]:
import numpy as np # importing numpy for numerical computation
from sklearn.datasets import load_boston # here we are using sklearn's boston dataset
from sklearn.metrics import mean_squared_error # importing mean_squared_error metric

import random
from sklearn.tree import DecisionTreeRegressor

boston = load_boston()
x=boston.data #independent variables
y=boston.target #target variable

print("x.shape ", x.shape)
print("y.shape ", y.shape)

x.shape  (506, 13)
y.shape  (506,)


In [2]:
x[:5]

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01,
        6.5750e+00, 6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02,
        1.5300e+01, 3.9690e+02, 4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
        6.4210e+00, 7.8900e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02,
        1.7800e+01, 3.9690e+02, 9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
        7.1850e+00, 6.1100e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02,
        1.7800e+01, 3.9283e+02, 4.0300e+00],
       [3.2370e-02, 0.0000e+00, 2.1800e+00, 0.0000e+00, 4.5800e-01,
        6.9980e+00, 4.5800e+01, 6.0622e+00, 3.0000e+00, 2.2200e+02,
        1.8700e+01, 3.9463e+02, 2.9400e+00],
       [6.9050e-02, 0.0000e+00, 2.1800e+00, 0.0000e+00, 4.5800e-01,
        7.1470e+00, 5.4200e+01, 6.0622e+00, 3.0000e+00, 2.2200e+02,
        1.8700e+01, 3.9690e+02, 5.3300e+00]])

### Generating samples

In [3]:
def generating_samples(input_data, target_data): 
  
  # random.choice to generate random indices without replacement
  # selecting 303 random row indices from the input_data, without replacement
  rows_selected = np.random.choice(len(input_data), 303, replace=False)
  
  # now we will replicate 203 points from above selected rows
  # Replacing Rows => Extracting 206 reandom row indices from the abvoe rows_selected
  rows_203_extracted_from_rows_selected = np.random.choice(rows_selected, 203, replace=False)
  
  # Now get 3 to 13 random column indices from input_data
  number_of_columns_to_select = random.randint(3, 13)
  columns_selected = np.array(random.sample(range(0, 13), number_of_columns_to_select ))
  
  sample_data = input_data[rows_selected[:, None], columns_selected]
  
  target_of_sample_data = target_data[rows_selected]
  
  # Now Replication of Data for 203 data points out of 303 selected points
  
  replicated_203_sample_data_points = input_data[rows_203_extracted_from_rows_selected[:, None], columns_selected ]
  
  target_203_replicated_sample_data = target_data[rows_203_extracted_from_rows_selected]
  
  # Concatenating data
  
  final_sample_data = np.vstack((sample_data, replicated_203_sample_data_points ))
  
  final_target_data = np.vstack((target_of_sample_data.reshape(-1, 1), target_203_replicated_sample_data.reshape(-1, 1) ))
  
  return final_sample_data, final_target_data, rows_selected, columns_selected
  

<font color='cyan'> <b> Grader function - 1 </b> </fongt>

In [4]:
def grader_samples(a,b,c,d):
    length = (len(a)==506  and len(b)==506)
    sampled = (len(a)-len(set([str(i) for i in a]))==203)
    rows_length = (len(c)==303)
    column_length= (len(d)>=3)
    assert(length and sampled and rows_length and column_length)
    return True
  
a,b,c,d = generating_samples(x, y)
print(a.shape)
print(b.shape)
print(c.shape)
print(d.shape)
grader_samples(a,b,c,d)

(506, 9)
(506, 1)
(303,)
(9,)


True

In [5]:
# Use generating_samples function to create 30 samples 
# store these created samples in a list
list_input_data =[]
list_output_data =[]
list_selected_row= []
list_selected_columns=[]

for i in range (0, 30):
  a, b, c, d = generating_samples(x, y)
  list_input_data.append(a)
  list_output_data.append(b)
  list_selected_row.append(c)
  list_selected_columns.append(d)

<font color='cyan'> <b>Grader function - 2 </b></font>

In [6]:
def grader_30(a):
    assert(len(a)==30 and len(a[0])==506)
    return True
grader_30(list_input_data)

True

In [7]:
list_of_all_models_decision_tree = []
for i in range(0, 30):
  model_i = DecisionTreeRegressor(max_depth=None)
  model_i.fit(list_input_data[i], list_output_data[i])
  list_of_all_models_decision_tree.append(model_i)

After getting predicted_y for each data point, we can use sklearns mean_squared_error to calculate the MSE between predicted_y and actual_y.

# Calculating MSE

In [8]:
from sklearn.metrics import mean_squared_error
from statistics import median

array_of_Y = []

for i in range(0, 30):
  data_point_i = x[:, list_selected_columns[i]]
  target_y_i = list_of_all_models_decision_tree[i].predict(data_point_i)
  array_of_Y.append(target_y_i)
  
  
predicted_array_of_target_y = np.array(array_of_Y)
predicted_array_of_target_y = predicted_array_of_target_y.transpose()

# print(predicted_array_of_target_y.shape)

# Now to calculate MSE, first calculate the Median of Predicted Y
# passing axis=1 will make sure the medians are computed along axis=1
median_predicted_y = np.median(predicted_array_of_target_y, axis=1)
median_predicted_y.shape

print("MSE : ", mean_squared_error(y, median_predicted_y ))

MSE :  0.020601008560538814


In [9]:
# First noting that our earlier definded variable list_selected_row and list_selected_columns 
# which has the list of selected rows and columns
# e.g. list_selected_row is a 2D array of the form [[], [], []...] each inner-array represnting selected row numbers
# for a specific sample. and len(list_selected_row) is 30 reprsenting the 30 samples we have selected for bootstrapping
# print("list_selected_row[10] ", list_selected_row[10])
# print(list_selected_columns)

y_predicted_oob_median_list = []

for i in range(0, 506):
  indices_for_oob_models = []
  
  # For each of i-th row I shall build a list, of sample size 30
  # ONLY condition being that this i-th row should not be part of the list_selected_row[i-th]
  # e.g. say for i = 469 and index_oob in below loop is 10 then 
  # list_selected_row[10] (which is an array of row-numbers) should not contain the 469-th row
  for index_oob in range(0, 30):
    if i not in list_selected_row[index_oob]:
      indices_for_oob_models.append(index_oob)
      
  y_predicted_oob_list = []
  
  for oob_model_index in indices_for_oob_models:
    model_oob = list_of_all_models_decision_tree[oob_model_index]
    
    row_oob = x[i]
    # print('oob_model_index ', oob_model_index)
    
    # Now extract ONLY those specific columns/featues that were selected during the bootstrapping
    x_oob_data_point = [row_oob[columns] for columns in list_selected_columns[oob_model_index] ]
    # print('np.array(x_oob_data_point) ', np.array(x_oob_data_point))
    x_oob_data_point = np.array(x_oob_data_point).reshape(1, -1)
    
    y_predicted_oob_data_point = model_oob.predict(x_oob_data_point)
    y_predicted_oob_list.append(y_predicted_oob_data_point)
    # 
  y_predicted_oob_list = np.array(y_predicted_oob_list)
  
  y_predicted_median = np.median(y_predicted_oob_list)
  y_predicted_oob_median_list.append(y_predicted_median)
  

def calculate_oob_score(num_rows):
  oob_score = 0
  for i in range(0, num_rows):
    oob_score += ((y[i] - y_predicted_oob_median_list[i] ) ** 2)
  final_oob_score = oob_score/506
  return final_oob_score

print("final_oob_score is ", calculate_oob_score(506))   
  

final_oob_score is  12.88396115502854


In [10]:
# Function to build the entire bootstrapping steps that we did above and
# Reurning from the function the MSE and oob score
def bootstrapping_and_oob(x, y):

  # Use generating_samples function to create 30 samples 
  # store these created samples in a list
  list_input_data =[]
  list_output_data =[]
  list_selected_row= []
  list_selected_columns=[]
  
  for i in range (0, 30):
    a, b, c, d = generating_samples(x, y)
    list_input_data.append(a)
    list_output_data.append(b)
    list_selected_row.append(c)
    list_selected_columns.append(d)
  
  # building regression trees
  list_of_all_models_decision_tree = []
  for i in range(0, 30):
    model_i = DecisionTreeRegressor(max_depth=None)
    model_i.fit(list_input_data[i], list_output_data[i])
    list_of_all_models_decision_tree.append(model_i)
  
  # calculating MSE
  array_of_Y = []

  for i in range(0, 30):
    data_point_i = x[:, list_selected_columns[i]]
    target_y_i = list_of_all_models_decision_tree[i].predict(data_point_i)
    array_of_Y.append(target_y_i)
    
    
  predicted_array_of_target_y = np.array(array_of_Y)
  predicted_array_of_target_y = predicted_array_of_target_y.transpose()

  # print(predicted_array_of_target_y.shape)

  # Now to calculate MSE, first calculate the Median of Predicted Y
  # passing axis=1 will make sure the medians are computed along axis=1
  median_predicted_y = np.median(predicted_array_of_target_y, axis=1)
  
  # And now the final MSE
  MSE = mean_squared_error(y, median_predicted_y )
  
  # Calculating OOB Score
  y_predicted_oob_median_list = []

  for i in range(0, 506):
    indices_for_oob_models = []
    
    # For each of i-th row I shall build a list of sample size 30
    # ONLY condition being that this ith row should not be part of
    # the list_selected_row
    for index_oob in range(0, 30):
      if i not in list_selected_row[index_oob]:
        indices_for_oob_models.append(index_oob)
        
    y_predicted_oob_list = []
    
    for oob_model_index in indices_for_oob_models:
      model_oob = list_of_all_models_decision_tree[oob_model_index]
      
      row_oob = x[i]
      # print('oob_model_index ', oob_model_index)
      
      x_oob_data_point = [row_oob[col] for col in list_selected_columns[oob_model_index] ]
      # print('np.array(x_oob_data_point) ', np.array(x_oob_data_point))
      x_oob_data_point = np.array(x_oob_data_point).reshape(1, -1)
      
      y_predicted_oob_data_point = model_oob.predict(x_oob_data_point)
      y_predicted_oob_list.append(y_predicted_oob_data_point)
      # 
    y_predicted_oob_list = np.array(y_predicted_oob_list)
    
    y_predicted_median = np.median(y_predicted_oob_list)
    y_predicted_oob_median_list.append(y_predicted_median)
    

  oob_score = 0

  for i in range(0, 506):
    # oob_score = (oob_score + (y[i] - y_predicted_oob_median_list[i] ) ** 2)
    # 13.828377285079045
    oob_score += (y[i] - y_predicted_oob_median_list[i] ) ** 2

  final_oob_score = oob_score/506
  
  return MSE, final_oob_score

print(bootstrapping_and_oob(x, y))
  

(0.07893375567065226, 10.495318261846007)


In [11]:
import scipy

x=boston.data #independent variables
y=boston.target #target variable

mse_boston_35_times_arr = []
oob_score_boston_35_times_arr = []

# Repeat Task 1 for 35 times, and for each iteration store the Train MSE and OOB score
for i in range(0, 35):
  mse, oob_score = bootstrapping_and_oob(x, y)
  mse_boston_35_times_arr.append(mse)
  oob_score_boston_35_times_arr.append(oob_score)


mse_boston_35_times_arr = np.array(mse_boston_35_times_arr)
oob_score_boston_35_times_arr = np.array(oob_score_boston_35_times_arr)

confidence_level = 0.95
degrees_of_freedom = 34 # sample.size - 1

mean_of_sample_mse_35 = np.mean(mse_boston_35_times_arr)
standard_error_of_sample_mse_35 = scipy.stats.sem(mse_boston_35_times_arr)


# Per document - https://www.kite.com/python/answers/how-to-compute-the-confidence-interval-of-a-sample-statistic-in-python
# confidence_interval = scipy.stats.t.interval(confidence_level, degrees_freedom, sample_mean, sample_standard_error)
confidence_interval_mse_35 = scipy.stats.t.interval(confidence_level, degrees_of_freedom, mean_of_sample_mse_35, standard_error_of_sample_mse_35 )
print("confidence_interval_mse_35 ", confidence_interval_mse_35)

# Now calculate confidence inter for oob score
mean_of_sample_oob_score_35 = np.mean(oob_score_boston_35_times_arr)
standard_error_of_sample_oob_score_35 = scipy.stats.sem(oob_score_boston_35_times_arr)

confidence_interval_oob_score_35 = scipy.stats.t.interval(confidence_level, degrees_of_freedom, mean_of_sample_oob_score_35, standard_error_of_sample_oob_score_35 )
print("confidence_interval_oob_score_35 ", confidence_interval_oob_score_35)


confidence_interval_mse_35  (0.07065402698924403, 0.15383817239118291)
confidence_interval_oob_score_35  (13.936187851151747, 14.966849102743751)


In [12]:
def predict_y_given_x_bootstrap(x_query):
  y_predicted_array_30_sample = []
  
  for i in range(0, 30):
    model_i = list_of_all_models_decision_tree[i]
    # Extract x for ith data point with specific number of featues from list_selected_columns
    x_data_point_i = [x_query[column] for column in list_selected_columns[i]]
    x_data_point_i = np.array(x_data_point_i).reshape(1, -1)
    y_predicted_i = model_i.predict(x_data_point_i)
    y_predicted_array_30_sample.append(y_predicted_i)
    
  y_predicted_array_30_sample = np.array(y_predicted_array_30_sample)
  y_predicted_median = np.median(y_predicted_array_30_sample)
  return y_predicted_median


xq= [0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60] 
y_predicted_for_xq = predict_y_given_x_bootstrap(xq)
y_predicted_for_xq    

19.4